In [1]:
import pandas as pd
from pymongo import MongoClient
import pprint
import re
import numpy as np
import folium
import os
import requests
from dotenv import load_dotenv
load_dotenv()

if not "KEY" in os.environ:
    raise ValueError("You should pass a KEY")
KEY = os.environ["KEY"]

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [3]:
companies = db.data_companies_clean.find({ 
    "total_money_raised":{
        "$gte": 1000000
    }
}
)  

#Primero filtro en las ganancias. Queremos empresas con al menos 1000000 de ingresos

In [4]:
data_companies = pd.DataFrame(companies)

KeyboardInterrupt: 

In [37]:
data_companies[data_companies['total_money_raised']==data_companies['total_money_raised'].max()]

#Para hacer una primera aproximación veo qué empresa tiene más dinero.

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country
66,5dd5b5d79428d15f28d17207,HootSuite,social,370.0,2008,190000000,"{'type': 'Point', 'coordinates': [-123.0952381...",49.285173,-123.095238,Vancouver,CAN


In [38]:
list_geo = []
for e in range(len(data_companies)):
    list_geo.append(data_companies.geoDescription[e]) #Lista con todas las geoDescription

In [7]:
#Función para saber el número de empresas que hay alrededor de cada una de ellas.

def findNear(list_geo, radio_meters):
        geopoint = list_geo
        return list(db.data_companies_clean.find({
        "geoDescription": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }
       }
    }
    )
    ) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  findNear(list_geo[i], radio_meters)
    list_offices.append(num_offices)
    list_number_offices.append(len(num_offices))
data_companies['number of offices near'] = list_number_offices

OperationFailure: error processing query: ns=companies.data_companies_cleanTree: GEONEAR  field=geoDescription maxdist=5000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query

In [8]:
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country
0,5dd5b5d79428d15f28d1719b,Bragster,tech,10.0,2007,3500000,"{'type': 'Point', 'coordinates': [-0.0766034, ...",51.522007,-0.076603,London,GBR
1,5dd5b5d79428d15f28d1719c,Zemanta,tech,30.0,2007,7350000,"{'type': 'Point', 'coordinates': [-0.0934254, ...",51.514157,-0.093425,London,GBR
2,5dd5b5d79428d15f28d1719d,eRepublik,tech,30.0,2007,2750000,"{'type': 'Point', 'coordinates': [-3.6952896, ...",40.430940,-3.695290,Madrid,ESP
3,5dd5b5d79428d15f28d171a1,Wakoopa,tech,8.0,2007,1000000,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Amsterdam,NLD
4,5dd5b5d79428d15f28d171a2,ModiFace,tech,35.0,2007,4250000,"{'type': 'Point', 'coordinates': [-79.388279, ...",43.666953,-79.388279,Toronto,CAN


In [9]:
data_companies['money by offices'] = data_companies['total_money_raised']/data_companies['number of offices near']

#Media de dinero por oficinas y creo nueva columna

KeyError: 'number of offices near'

In [10]:
def sumEmployees(list_offices):
    list_employee = []
    res=0
    for company in list_offices:
        lst=[]
        for i in company:
            lst.append(i['number_of_employees'])
        res+=i['number_of_employees']
        list_employee.append(lst)
    result = [sum(b) for b in list_employee]
    return result

#función para calcular la suma de empleados por oficina

In [11]:
data_companies['employee by offices'] = sumEmployees(list_offices)/data_companies['number of offices near']

KeyError: 'number of offices near'

In [12]:
data_companies = data_companies[data_companies['number of offices near'] > 1]

#Quiero que al menos haya una oficina cerca

KeyError: 'number of offices near'

In [13]:
#Unifico los valores para poder hacer el ranking 

data_companies['Rank_employee'] = data_companies['employee by offices'].rank()
data_companies['Rank_money'] = data_companies['money by offices'].rank()
data_companies['Rank_offices'] = data_companies['number of offices near'].rank()

data_companies.head()

KeyError: 'employee by offices'

In [14]:
data_companies['ranking_total'] =  data_companies['Rank_money']*0.8 + data_companies['Rank_employee']*0.6 + data_companies['Rank_offices']*0.4

#Hago un ranking total

KeyError: 'Rank_money'

In [15]:
data_companies = data_companies.sort_values(['ranking_total'], ascending=False)

#Ordeno por ranking total

KeyError: 'ranking_total'

In [16]:
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch"

In [17]:
res = requests.get("{}/json?location=49.285173,-123.095238&radius=1500&type=school&key={}".format(BASE_URL, KEY)).json()

In [18]:
#Con la Api de google busco las escuelas cerca sobre la empresa localizada en Vancouver que es la primera según ranking

coordinates_schools_latitude = []
coordinates_schools_longitude = []
name_schools = []
for i in range(len(res['results'])):
    coordinates_schools_latitude.append(res['results'][i]['geometry']['location']["lat"])
    coordinates_schools_longitude.append(res['results'][i]['geometry']['location']["lng"])
    name_schools.append(res['results'][i]['name'])
print(coordinates_schools_latitude, coordinates_schools_longitude, name_schools)

[] [] []


In [19]:
dicc = {'latitude_school':coordinates_schools_latitude,
       'longitude_school': coordinates_schools_longitude,
       'name_school': name_schools}

In [20]:
dataframe_schools = pd.DataFrame(dicc)

In [21]:
display(dataframe_schools)

,latitude_school,longitude_school,name_school


In [22]:
data_companies.head()

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,geoDescription,latitude,longitude,city,country
0,5dd5b5d79428d15f28d1719b,Bragster,tech,10.0,2007,3500000,"{'type': 'Point', 'coordinates': [-0.0766034, ...",51.522007,-0.076603,London,GBR
1,5dd5b5d79428d15f28d1719c,Zemanta,tech,30.0,2007,7350000,"{'type': 'Point', 'coordinates': [-0.0934254, ...",51.514157,-0.093425,London,GBR
2,5dd5b5d79428d15f28d1719d,eRepublik,tech,30.0,2007,2750000,"{'type': 'Point', 'coordinates': [-3.6952896, ...",40.430940,-3.695290,Madrid,ESP
3,5dd5b5d79428d15f28d171a1,Wakoopa,tech,8.0,2007,1000000,"{'type': 'Point', 'coordinates': [4.8948623, 5...",52.374523,4.894862,Amsterdam,NLD
4,5dd5b5d79428d15f28d171a2,ModiFace,tech,35.0,2007,4250000,"{'type': 'Point', 'coordinates': [-79.388279, ...",43.666953,-79.388279,Toronto,CAN


In [23]:
data_companies = data_companies.reset_index(drop=True)

In [24]:
data_companies.to_csv('./data_companies_select.csv')

In [25]:
res = requests.get("{}/json?location=49.285173,-123.095238&radius=1500&type=cafe&key={}&name=Starbucks".format(BASE_URL, KEY))

In [26]:
res = res.json()

In [27]:
coordinates_starbucks_latitude = []
coordinates_starbucks_longitude = []
for i in range(len(res['results'])):
    coordinates_starbucks_latitude.append(res['results'][i]['geometry']['location']["lat"])
    coordinates_starbucks_longitude.append(res['results'][i]['geometry']['location']["lng"])
print(coordinates_starbucks_latitude, coordinates_starbucks_longitude)

[] []


In [28]:
dicc_starbucks = {'latitude_starbucks':coordinates_starbucks_latitude,
       'longitude_starbucks': coordinates_starbucks_longitude}
dataframe_starbucks = pd.DataFrame(dicc_starbucks)
display(dataframe_starbucks)

,latitude_starbucks,longitude_starbucks


In [29]:
#Mapa con toda las compañías

In [30]:
map_companies = folium.Map(location=[51.510880, -0.141897], width=750, height=500, zoom_start=2)
for index, row in data_companies.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=3,
                        popup="Country: {}. Money: {}. Name:{}".format(row['country'], row['total_money_raised'], row['name']),
                        fill_color="#F35C50", # divvy color
                       ).add_to(map_companies)

map_companies.save('map_companies.html')
map_companies

In [31]:
#Mapa con las empresas del radio en la zona, escuelas cercanas (en rojo), Starbucks (verde) y la calle donde 
#sería interesante localizar la empresa

In [32]:
map_vancouver = folium.Map(location=[49.285173, -123.095238], width=750, height=500, zoom_start=15)
for index, row in data_companies.iterrows():
    folium.Marker([row['latitude'], row['longitude']],
                        radius=7,
                        popup="Name company: {}. Money: {}. Employees{}".format(row['name'], row['total_money_raised'], row['number_of_employees']),
                        icon=folium.Icon(color='red', icon='info-sign'),
                        fill_color="#F45649", 
                       ).add_to(map_vancouver)
for index, row in dataframe_schools.iterrows():
    folium.Marker([row['latitude_school'], row['longitude_school']],
                        radius=4,
                        popup="School: {}".format(row['name_school']),
                        icon=folium.Icon(icon='cloud'),
                        fill_color="#F35C50", # divvy color
                       ).add_to(map_vancouver)
for index, row in dataframe_starbucks.iterrows():
    folium.Marker([row['latitude_starbucks'], row['longitude_starbucks']],
                        radius=4,
                        icon=folium.Icon(color='darkgreen',icon='info-sign'),
                        fill_color="#F35C50", # divvy color
                       ).add_to(map_vancouver)
folium.Marker([49.282223,-123.109113],
                        radius=4,
                        icon=folium.Icon(color='pink',icon='info-sign'),
                        popup="Our company",
                        fill_color="#F35C50", # divvy color
                       ).add_to(map_vancouver)
folium.Circle([49.282223, -123.109113],
                    radius=300
                   ).add_to(map_vancouver)

map_vancouver.save('map_vancouver.html')
map_vancouver